
https://or.stackexchange.com/questions/9907/optimal-to-do-list-scheduling-in-python-using-pyomo

J'essaie de planifier de manière optimale une série de tâches avec des durées fixes entre 9h et 17h dans la journée (je suppose un peu comme un problème de sac à dos contraint ou un problème de planification de travail). J'ai divisé la journée en incréments de 15 minutes et j'ai supposé que ma fenêtre de travail était de 8 heures (incréments de 32 x 15 minutes). Dans les colonnes de la matrice de sortie se trouvent les tâches dans l'ordre de la tâche 1 à la tâche 4. Si une cellule est 1, cela signifie que la tâche a été planifiée pour cet intervalle particulier.

Mon code résout en quelque sorte le problème, mais il ne traite pas une tâche comme ayant une durée fixe. Ainsi, dans mon exemple, la tâche 4 s'exécute pendant 3 heures, alors qu'en réalité elle ne devrait pas s'exécuter du tout car il n'y a pas assez de temps pour intégrer les 4 heures complètes.

J'ai toute une gamme de contraintes supplémentaires que j'aimerais introduire (par exemple, les priorités des tâches, la fixation de certaines tâches à des moments précis, etc.), mais pour l'instant, je voulais rester simple. J'aimerais également recevoir des suggestions ou un exemple de code pour encore mieux aborder ce problème (d'autres options en dehors de la programmation linéaire sont également les bienvenues).



In [3]:
from pyomo.environ import *

# Inputs
task_list      = ['Task 1', 'Task 2', 'Task 3', 'Task 4']
durations = [2,1,2,4]

Intervals = 32
Tasks = len(durations)

# Construct model variables
model = ConcreteModel()
model.Intervals = range(Intervals)
model.Tasks = range(Tasks)
model.flag = Var( model.Intervals, model.Tasks, within=Integers)
model.x = Var( model.Intervals, model.Tasks, within=Binary )

# Set objective
model.obj = Objective(expr = sum(model.x[n,m] * 0.25 for n in model.Intervals for m in model.Tasks ), sense = maximize )

# Set constraints
model.row_constraint = ConstraintList()
for n in model.Intervals:
    model.row_constraint.add(sum( model.x[n,m] * 0.25 for m in model.Tasks) <= 0.25)

model.column_constraint = ConstraintList()
for m in model.Tasks:
    model.column_constraint.add(sum( model.x[n,m] * 0.25 for n in model.Intervals ) <= durations[m])

model.flag_making = ConstraintList()
for m in model.Tasks:
    for n in model.Intervals:
        if n == 0:
           model.flag_making.add(model.x[n,m] - 0 == model.flag[n,m]) 
        elif n == Intervals-1:
            model.flag_making.add(0 - model.x[n,m] == model.flag[n,m])
        else:
            model.flag_making.add(model.x[n,m] - model.x[n-1,m] == model.flag[n,m])
for m in model.Tasks:
    model.column_constraint.add(sum( model.x[n,m] * 0.25 for n in model.Intervals ) <= durations[m])

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Post processing
outputMatrix = [[value(model.x[Intervals,Tasks]) for Tasks in model.Tasks] for Intervals in model.Intervals]
df = pd.DataFrame(outputMatrix, columns = task_list)

print("\nObjective Value:")
print(model.obj())

df

    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'